**Kelly Putnam**

Spring 2021

CS 251: Data Analysis and Visualization

Project 6: Supervised learning

In [3]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])
plt.rcParams.update({'font.size': 20})

np.set_printoptions(suppress=True, precision=5)

# Automatically reload external modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Task 3: Preprocess full spam email dataset 

Before you build a Naive Bayes spam email classifier, run the full spam email dataset through your preprocessing code.

Download and extract the full **Enron** emails (*zip file should be ~29MB large*). You should see a base `enron` folder, with `spam` and `ham` subfolders when you extract the zip file (these are the 2 classes).

Run the test code below to check everything over.

### 3a) Preprocess dataset

In [5]:
import email_preprocessor as epp

#### Test `count_words` and `find_top_words`

In [6]:
word_freq, num_emails = epp.count_words()

In [7]:
print(f'You found {num_emails} emails in the datset. You should have found 32625.')

You found 32625 emails in the datset. You should have found 32625.


In [8]:
top_words, top_counts = epp.find_top_words(word_freq)
print(f"Your top 5 words are\n{top_words[:5]}\nand they should be\n['the', 'to', 'and', 'of', 'a']")
print(f"The associated counts are\n{top_counts[:5]}\nand they should be\n[277459, 203659, 148873, 139578, 111796]")

Your top 5 words are
['the', 'to', 'and', 'of', 'a']
and they should be
['the', 'to', 'and', 'of', 'a']
The associated counts are
[277459, 203659, 148873, 139578, 111796]
and they should be
[277459, 203659, 148873, 139578, 111796]


### 3b) Make train and test splits of the dataset

Here we divide the email features into a 80/20 train/test split (80% of data used to train the supervised learning model, 20% we withhold and use for testing / prediction).

In [9]:


features, y = epp.make_feature_vectors(top_words, num_emails)

In [11]:
np.random.seed(0)

x_train, y_train, inds_train, x_test, y_test, inds_test = epp.make_train_test_sets(features, y)



In [12]:
print('Shapes for train/test splits:')
print(f'Train {x_train.shape}, classes {y_train.shape}')
print(f'Test {x_test.shape}, classes {y_test.shape}')
print('\nThey should be:\nTrain (26100, 200), classes (26100,)\nTest (6525, 200), classes (6525,)')

Shapes for train/test splits:
Train (26100, 200), classes (26100,)
Test (6525, 200), classes (6525,)

They should be:
Train (26100, 200), classes (26100,)
Test (6525, 200), classes (6525,)


### 3c) Save data in binary format

It adds a lot of overhead to have to run through your raw email -> train/test feature split every time you wanted to work on your project! In this step, you will export the data in memory to disk in a binary format. That way, you can quickly load all the data back into memory (directly in ndarray format) whenever you want to work with it again. No need to parse from text files!

- Use numpy's `save` function to make six files in `.npy` format (e.g. `email_train_x.npy`, `email_train_y.npy`, `email_train_inds.npy`, `email_test_x.npy`, `email_test_y.npy`, `email_test_inds.npy`).

In [13]:
np.save('email_train_x', x_train)
np.save('email_train_y', y_train)
np.save('email_train_inds', inds_train)
np.save('email_test_x', x_test)
np.save('email_test_y', y_test)
np.save('email_test_inds', inds_test)

## Task 4: Naive Bayes Classifier

After finishing your email preprocessing pipeline, implement the one other supervised learning algorithm we we will use to classify email, **Naive Bayes**.

### 4a) Implement Naive Bayes

In `naive_bayes.py`, implement the following methods:
- Constructor
- `train(data, y)`: Train the Naive Bayes classifier so that it records the "statistics" of the training set: class priors (i.e. how likely an email is in the training set to be spam or ham?) and the class likelihoods (the probability of a word appearing in each class — spam or ham).
- `predict(data)`: Combine the class likelihoods and priors to compute the posterior distribution. The predicted class for a test sample is the class that yields the highest posterior probability.
- `accuracy(y, y_pred)`: The usual definition :)


#### Bayes rule ingredients: Priors and likelihood (`train`)

To compute class predictions (probability that a test example belong to either spam or ham classes), we need to evaluate **Bayes Rule**. This means computing the priors and likelihoods based on the training data.

**Prior:** $$P_c = \frac{N_c}{N}$$ where $P_c$ is the prior for class $c$ (spam or ham), $N_c$ is the number of training samples that belong to class $c$ and $N$ is the total number of training samples.

**Likelihood:** $$L_{c,w} = \frac{N_{c,w} + 1}{N_{c} + M}$$ where
- $L_{c,w}$ is the likelihood that word $w$ belongs to class $c$ (*i.e. what we are solving for*)
- $N_{c,w}$ is the total count of **word $w$** in emails that are only in class $c$ (*either spam or ham*)
- $N_{c}$ is the total number of **all words** that appear in emails of the class $c$ (*total number of words in all spam emails or total number of words in all ham emails*)
- $M$ is the number of features (*number of top words*).

#### Bayes rule ingredients: Posterior (`predict`)

To make predictions, we now combine the prior and likelihood to get the posterior:

**Log Posterior:** $$Log(\text{Post}_{i, c}) = Log(P_c) + \sum_{j \in J_i}x_{i,j}Log(L_{c,j})$$

 where
- $\text{Post}_{i,c}$ is the posterior for class $c$ for test sample $i$(*i.e. evidence that email $i$ is spam or ham*). We solve for its logarithm.
- $Log(P_c)$ is the logarithm of the prior for class $c$.
- $x_{i,j}$ is the number of times the jth word appears in the ith email.
- $Log(L_{c,j})$: is the log-likelihood of the jth word in class $c$.

In [14]:
from naive_bayes_multinomial import NaiveBayes


#### Test `train`

In [15]:
num_test_classes = 4
np.random.seed(0)
data_test = np.random.random(size=(100, 6))
y_test = np.random.randint(low=0, high=num_test_classes, size=(100,))

nbc = NaiveBayes(num_classes=num_test_classes)
nbc.train(data_test, y_test)

print(f'Your class priors are: {nbc.class_priors}\nand should be          [0.24 0.26 0.25 0.25].')
print(f'Your class likelihoods shape is {nbc.class_likelihoods.shape} and should be (4, 6).')
print(f'Your likelihoods are:\n{nbc.class_likelihoods}')


test_likelihoods = np.array([[0.15116, 0.18497, 0.17571, 0.1463 , 0.16813, 0.17374],
       [0.16695, 0.17437, 0.15742, 0.16887, 0.15677, 0.17562],
       [0.14116, 0.1562 , 0.19651, 0.17046, 0.17951, 0.15617],
       [0.18677, 0.18231, 0.15884, 0.12265, 0.16755, 0.18187]])
print(f'and should be\n{test_likelihoods}')

Your class priors are: [0.24 0.26 0.25 0.25]
and should be          [0.24 0.26 0.25 0.25].
Your class likelihoods shape is (4, 6) and should be (4, 6).
Your likelihoods are:
[[0.15116 0.18497 0.17571 0.1463  0.16813 0.17374]
 [0.16695 0.17437 0.15742 0.16887 0.15677 0.17562]
 [0.14116 0.1562  0.19651 0.17046 0.17951 0.15617]
 [0.18677 0.18231 0.15884 0.12265 0.16755 0.18187]]
and should be
[[0.15116 0.18497 0.17571 0.1463  0.16813 0.17374]
 [0.16695 0.17437 0.15742 0.16887 0.15677 0.17562]
 [0.14116 0.1562  0.19651 0.17046 0.17951 0.15617]
 [0.18677 0.18231 0.15884 0.12265 0.16755 0.18187]]


#### Test `predict`

In [16]:
num_test_classes = 4
np.random.seed(0)
data_train = np.random.randint(low=0, high=num_test_classes, size=(100, 10))
data_test = np.random.randint(low=0, high=num_test_classes, size=(15, 10))
y_test = np.random.randint(low=0, high=num_test_classes, size=(100,))

nbc = NaiveBayes(num_classes=num_test_classes)
nbc.train(data_train, y_test)
test_y_pred = nbc.predict(data_test)

print(f'Your predicted classes are\n{test_y_pred}\nand should be\n[3 0 3 1 0 1 1 3 0 3 0 2 0 2 1]')

Your predicted classes are
[3 0 3 1 0 1 1 3 0 3 0 2 0 2 1]
and should be
[3 0 3 1 0 1 1 3 0 3 0 2 0 2 1]


### 4b) Spam filtering

Let's start classifying spam email using the Naive Bayes classifier.

- Use `np.load` to load in the train/test split that you created last week.
- Use your Naive Bayes classifier on the Enron email dataset!

**Question 7:** Print out the accuracy that you get on the test set with Naive Bayes. It should be roughly 89%.

In [17]:
import email_preprocessor as ep

In [18]:
# training data
x_train = np.load('data/email_train_x.npy')
y_train = np.load('data/email_train_y.npy')
train_inds = np.load('data/email_train_inds.npy')

# testing data
x_test = np.load('data/email_test_x.npy')
y_test = np.load('data/email_test_y.npy')
test_inds = np.load('data/email_test_inds.npy')

# finding accuracy
newnbc = NaiveBayes(num_classes=2)
newnbc.train(x_train, y_train)
y_pred = newnbc.predict(x_test)
print(newnbc.accuracy(y_test, y_pred))

0.8895019157088122


### 4c) Confusion matrix

To get a better sense of the errors that the Naive Bayes classifer makes, you will create a confusion matrix. 

- Implement `confusion_matrix` in `naive_bayes.py`.
- Print out a confusion matrix of the spam classification results.

**Debugging guidelines**:
1. The sum of all numbers in your 2x2 confusion matrix should equal the number of test samples (6525).
2. The sum of your spam row should equal the number of spam samples in the test set (3193)
3. The sum of your ham row should equal the number of spam samples in the test set (3332)

In [19]:
mat = newnbc.confusion_matrix(y_test, y_pred)
print(mat)
# print(np.sum(mat[0,:]))
# print(np.sum(mat[1,:]))

[[3025.  168.]
 [ 553. 2779.]]


**Question 8:** Interpret the confusion matrix, using the convention that positive detection means spam (*e.g. a false positive means classifying a ham email as spam*). What types of errors are made more frequently by the classifier? What does this mean (*i.e. X (spam/ham) is more likely to be classified than Y (spam/ham) than the other way around*)?

**Reminder:** Look back and make sure you are clear on which class indices correspond to spam/ham.

**Answer 8:** According to the top row, there were 3025 correct spam classifications (meaning 3025 times did the prediction for spam match up with the actual class), and there were 168 times that they were predicted to be spam but were not actually spam. Similarly, in the bottom row, there were 553 incorrect ham predictions (553 times was something predicted to be ham when it was actually spam) and 2779 correct ham predictions.

This means that ham is far more likely to be predicted as spam, when it is actually ham, than the other way around. However, in both cases, the correct predictions far exceeded the false ones.

### 4d) Investigate the misclassification errors

Numbers are nice, but they may not the best for developing your intuition. Sometimes, you want to see what an misclassification *actually looks like* to help you improve your algorithm. Here, you will take a false positive and a false negative misclassification and retrieve the actual text of the email so see which emails produced the error.

- Determine the index of the **FIRST** false positive and false negative misclassification — i.e. 2 indices in total. Remember to use your `test_inds` array to look up the index of the emails BEFORE shuffling happened.
- Implement the function `retrieve_emails` in `email_preprocessor.py` to return the string of the raw email at the error indices.
- Call your function to print out the two emails that produced misclassifications.

**Question 9:** Does it seem reasonable that each email message was misclassified? Why?

**Answer 9:** So the first email was predicted to be ham when it was actually spam, which makes sense if you look at the email. I'm sure a lot of the fancy medical words in the email might seem like ham, but as a human I would look at this and classify it as spam. The second email was predicted to be spam when it was actually ham. This makes sense because spam usually is trying to sell you something and includes the word "prices", etc, which this email is doing so I'm sure the email misclassified it based off of that.

In [20]:
# finding first misclassification ; it's 1 if it's ham, 0 if spam

for i in range(10):
    if y_test[i] == 0 and y_pred[i] == 1:
        print( 'predicted ham, actually spam at ind',i)
    if y_test[i] == 1 and y_pred[i] == 0:
        print('predicted spam, actually ham at ind',i)





# see that it's index 2 and 6
falsepos, falseneg = ep.retrieve_emails([2,6])

print('PREDICTED HAM, ACTUALLY SPAM:',falsepos,'\n')
print('PREDICTED SPAM, ACTUALLY HAM:',falseneg)

predicted spam, actually ham at ind 2
predicted ham, actually spam at ind 6
predicted ham, actually spam at ind 9
PREDICTED HAM, ACTUALLY SPAM: Subject: fwd : have meds . xa _ n _ ax . vali . u . m . v | ^ codin . v ! agr @ . folavsni if a physician make a large incision with an operating knife and cure it , or if he open a tumor ( over the eye ) with an operating knife , and saves the eye , he shall receive ten shekels in money . we go out of our way to insure that your health care needs are attended to in a one - stop process . levitra 5 pills - 20 mg tablets - - $ 129 . 99 ambien 60 pills - 10 mg tablets - - $ 249 . 99 ambien 60 pills - 5 mg tablets - - $ 259 . 99 carisoprodol ( soma ) 90 pills - 350 mg tablets - $ 129 . 99 we accept almost every form of payment . because you can add more to your life . shop now . . for phone orders please call us at 1 - 415 - 462 - 2965 or 1 - 646 - 837 - 0772 ( monday - saturday , 09 : 30 am to 10 : 00 pm est ) and patience tame to sufferance bide

## Task 5: Comparison with KNN


- Run a similar analysis to what you did with Naive Bayes above. When computing accuracy on the test set, you may want to reduce the size of the test set (e.g. to the first 500 emails in the test set).
- Copy-paste your `confusion_matrix` method into `knn.py` so that you can run the same analysis on a KNN classifier.

In [21]:
from knn import KNN

In [22]:
knn_clsfr = KNN(num_classes=2)

knn_clsfr.train(x_train, y_train)
y_pred = knn_clsfr.predict(x_test, 5)
knnacc = knn_clsfr.accuracy(y_test[:500], y_pred[:500])
knnconfmat = knn_clsfr.confusion_matrix(y_test, y_pred)

print('ACCURACY:',knnacc,'\n')
print('CONFUSION MATRIX:',knnconfmat)


ACCURACY: 0.932 

CONFUSION MATRIX: [[3026.  167.]
 [ 369. 2963.]]


**Question 10:** What accuracy did you get on the test set (potentially reduced in size)?

**Question 11:** How does the confusion matrix compare to that obtained by Naive Bayes (*If you reduced the test set size, keep that in mind*)?

**Question 12:** Briefly describe at least one pro/con of KNN compared to Naive Bayes on this dataset.

**Question 13:** When potentially reducing the size of the test set here, why is it important that we shuffled our train and test set?

**Answer 10:** I got an accuracy of 0.932 on the reduced test set of 500 emails. 

**Answer 11:** The confusion matrix from KNN actually was more successful; there were less false classifications overall. However, this definitely could have to do with reducing the test size. 

**Answer 12:** KNN is much slower because of all the distance calculations it takes (looping through every single data point multiple times). This makes it less ideal for larger datasets; however, for smaller datasets, KNN can be better because you don't have to deal with Laplacian smoothing and taking logarithms in order to avoid zero/small probabilities.

**Answer 13:** Because we reduced the size of the test set, we want to make sure we arent being biased and only using the first 500 emails in the original set instead of actually getting a random sample from all over the dataset. 

## Extensions

### 0. Classify your own datasets

- Find datasets that you find interesting and run classification on them using your KNN algorithm (and if applicable, Naive Bayes). Analysis the performance of your classifer.

In [68]:
from knn import KNN
from naive_bayes_multinomial import NaiveBayes

'''TESTING USING SKEWNESS AND VARIANCE'''

# loading data and splitting into testing and training
banknote_data = np.loadtxt('banknote.csv', skiprows=1, delimiter=',')
import email_preprocessor as epp
x_train, y_train, inds_train, x_test, y_test, inds_test = epp.make_train_test_sets(banknote_data[:,0:2], banknote_data[:,4], test_prop=0.2, shuffle=True)

# training
n_classes = 2
bank_clsfr = KNN(num_classes=n_classes)
bank_clsfr.train(x_train, y_train)

# testing with KNN
banknote_y_pred = bank_clsfr.predict(x_test, 10)
knnbankacc = bank_clsfr.accuracy(y=y_test ,y_pred=banknote_y_pred)
print('KNN accuracy using Variance v. Skewness was:',knnbankacc)



'''TESTING USING ENTROPY AND KURTOSIS'''
x_train, y_train, inds_train, x_test, y_test, inds_test = epp.make_train_test_sets(banknote_data[:,2:4], banknote_data[:,4], test_prop=0.2, shuffle=True)

# training
n_classes = 2
bank_clsfr = KNN(num_classes=n_classes)
bank_clsfr.train(x_train, y_train)

# testing with KNN
banknote_y_pred = bank_clsfr.predict(x_test, 7)
knnbankacc = bank_clsfr.accuracy(y=y_test ,y_pred=banknote_y_pred)
print('KNN accuracy using Entropy v. Kurtosis was:',knnbankacc)



KNN accuracy using Variance v. Skewness was: 0.9306569343065694
KNN accuracy using Entropy v. Kurtosis was: 0.7481751824817519


One dataset I found to be very interesting was one that decided whether or not banknotes were authentic or fake given a close-up image of them; there were 4 features other than class (authentic or nonauthentic) in the dataset, and I didn't know what any of them meant since they were all image-processing specific terms, so I decided to analyze the two most easy-to-understand features: skewness of the image as X (in context of image processing, this means the darker colored pixels are spread much differently, either less spread or more spread, than the lighter colored pixels) and variance of the image as Y (how different each pixel color is from the others) to see how they contributed to the classification of the banknote as authentic or fake. 

I tried out KNN classification first, trying out different values of k until I found one that yielded the highest accuracy. For Variance v. Skewness, the k didn't matter much; no matter what, the accuracy was very high at around 0.95. I found this value was most consistently high when I used a k of 10, however. Since this accuracy is very high, it shows that these two features very definitively affect whether or not a banknote is determined to be authentic or not. 

I decided to compare the other features in the dataset: as X, I tried kurtosis of image (kind of a measure of resolution/noise of an image, in that a higher kurtosis level corresponds to higher noise and resolution of an image). As Y, I tried entropy of image (measures the randomness of each pixel in an image). For this one, a higher k, such as 20, affected the accuracy by moving it down about 0.1, so settled on k=7 as the most accurate one, yielding about 0.75 every time I ran the classification. This is less accurate than the two other features, so I would say that kurtosis and entropy contribute less definitively to the classification of banknotes as authentic or not.

All in all, this proved that image processing, when the right features are analyzed, can be hugely important to determining whether or not a banknote is authentic or not.


### 1. Better text preprocessing

- If you look at the top words extracted from the email dataset, many of them are common "stop words" (e.g. a, the, to, etc.) that do not carry much meaning when it comes to differentiating between spam vs. non-spam email. Improve your preprocessing pipeline by building your top words without stop words. Analyze performance differences.

In [24]:
import email_preprocessor as epp
word_freq, num_emails = epp.count_words()

print('WITH STOP WORDS:')
top_words, top_counts = epp.find_top_words(word_freq, no_stop_words = False)
features, y = epp.make_feature_vectors(top_words, num_emails)
x_train, y_train, inds_train, x_test, y_test, inds_test = epp.make_train_test_sets(features, y)
newnbc = NaiveBayes(num_classes=2)
newnbc.train(x_train, y_train)
y_pred = newnbc.predict(x_test)
print(newnbc.accuracy(y_test, y_pred),'\n')

print('WITH NO STOP WORDS:')
top_words, top_counts = epp.find_top_words(word_freq, no_stop_words = True)
features, y = epp.make_feature_vectors(top_words, num_emails)
x_train, y_train, inds_train, x_test, y_test, inds_test = epp.make_train_test_sets(features, y)
newnbc = NaiveBayes(num_classes=2)
newnbc.train(x_train, y_train)
y_pred = newnbc.predict(x_test)
print(newnbc.accuracy(y_test, y_pred),'\n')


WITH STOP WORDS:

0.8905747126436782
WITH NO STOP WORDS:

0.8731034482758621


When I printed out the top words from the emails, I went through the list in order until I decided I had reached the end of the "stop words," which I decided was at index 32 or so -- after this index is the word "company," which does not seem like a common buzz word. I added a new boolean parameter to the top_words method, which defaults to False, but when set to true, filters out the stop words in the output for top words. I then found the Naive Bayes accuracy when run with top words with a false no_stop_words paramater, then compared it to a true no_stop_words parameter. I compared them by going through the whole test twice then comparing the accuracy of the naive_bayes_classifier for each; the accuracy for the no-stop-words was slightly lower, but definitely much more similar to the original accuracy than I thought it would be; I removed the most common words, after all. This is probably because these words appear in both spam and ham so often that they cancel each other out. 

### 2. Feature size

- Explore how the number of selected features for the email dataset influences accuracy and runtime performance.

### 3. Distance metrics
- Compare KNN performance with the $L^2$ and $L^1$ distance metrics

In [25]:
import numpy as np
from knn import KNN

In [30]:
from knn import KNN
# seeing difference between L2 and L1 distance
# loading necessary train and test dat
spiral_1_train = np.loadtxt('data/spiral_train_1.csv', skiprows=1, delimiter=',')
spiral_1_val = np.loadtxt('data/spiral_val_1.csv', skiprows=1, delimiter=',')
spiral_1_train_y = spiral_1_train[:, 2]
spiral_1_val_y = spiral_1_val[:, 2]
spiral_1_train = spiral_1_train[:, :2]
spiral_1_val = spiral_1_val[:, :2]
n_classes = 4
classifier = KNN(num_classes=n_classes)
classifier.train(spiral_1_train, spiral_1_train_y)
k = 2



# testing with l2 distance, as originally done 
spiral_1_y_pred = classifier.predict(spiral_1_val, k)
acc = classifier.accuracy(y=spiral_1_val_y, y_pred=spiral_1_y_pred)
print(f'Your accuracy with K=2 and L2 distance is {acc:.2f}')

# testing with new L1 distance method
spiral_1_y_pred_l1 = classifier.predict_cityblock(spiral_1_val, k)
acc = classifier.accuracy(y=spiral_1_val_y, y_pred=spiral_1_y_pred_l1)
print(f'Your accuracy with K=2 and L1 distance is {acc:.2f}')



Your accuracy with K=2 and L2 distance is 0.88
Your accuracy with K=2 and L1 distance is 0.25


The method in KNN that predicts the classes of the test values is also the one where you need to compute the distance to each of the k neighbors of each point, then see which class the most of them are in. When I originally implemented this method last week, I just used Euclidian distance because I thought it would be the most accurate. However, I decided to try another distance metric to see if it would make more sense; unfortunately, since I did not know what L1 distance was, I had to look it up. This is "Manhattan Cityblock" distance, which means it measures the sum of the magnitudes of the vectors in the x direction and y direction, since in Manhattan you can't cut across blocks and make diagonal shortcuts, as you can in L2 distance metrics. I implemented a new predict method, predict_cityblock, which calculates manhattan distance, then compared the accuracies of using both predict/distance methods. As I thought, the accuracy for L2 distance was 0.88 and the accuracy for L1 distance was 0.25, most likely because the distance should probably be measured radially (in one straight, direct line) to be more accurate. 

### 4. K-Fold Cross-Validation

- Research this technique and apply it to data and your KNN and/or Naive Bayes classifiers.

### 5. Email error analysis

- Dive deeper into the properties of the emails that were misclassified (FP and/or FN) by Naive Bayes or KNN. What is their word composition? How many words were skipped because they were not in the training set? What could plausibly account for the misclassifications?